In [27]:
import numpy as np
import math
import datetime

lambdaa = 80
lambdas = 200
taus = 200
taua = 200

def frameDevFunction(i, j, v):
    frameCostValue = min(math.pow(((j-i)-v),2), taus)
    return frameCostValue

def frameAccelCost(h, i, j):
    frameaccelerationcost = min(math.pow(((j-i) - (h-i)),2), taua)
    return frameaccelerationcost

def arrayPath(frameCount, extraDiag, v):
    
    #v is targetted speed up; v<=extraDiag
    print "Loading Array"
    errors = np.loadtxt('array.txt')
    print "Array Loaded"
    #Add our speed up cost function
    for i in range(0, frameCount):
        for j in range(i+1, i+extraDiag+2):
            if(j<frameCount and i<frameCount):
                errors[i,j] += lambdas * frameDevFunction(i, j, v)
    print "Speed Up Cost Added"
    
    #Array is populated now assess costs
    costmatrix = np.zeros((frameCount, frameCount))
    #First Pass of Populate
    for i in range(1,frameCount):
        for j in range(i+1, i+extraDiag+2):
            n1 = []    
            #Populate a new list based on the for loop
            for k in range(i-1, i-(extraDiag+2),-1):
                #Populate our list
                n1.insert(k, (errors[k, i] + lambdaa*frameAccelCost(k, i, j)))
            if(i< frameCount and j<frameCount):
                errors[i, j]=np.min(n1)               
                #Is this next line correct or should we not build on the prior result? Pseudo-code is ambiguous          
                costmatrix[i, j] = i - ((extraDiag+1) - np.argmin(n1))   
    print "Cost Matrix Calculated"
                         
                         
    #Second Pass Trace Back Min Cost
    #Multi-dimensional argmin to set initial values of s and d (basically the min value in the last few rows we can select)
    #This will do for now
    s = frameCount -3
    d = s + 1
    #What we need is to isolate a subset of the array then find the argmin for this
    
    p = []
    while(s>1):
        #Prepend to List and work our way back up through the cost matrix
        p.insert(0, s)
        b = costmatrix[s, d]
        d = s
        s = b
    np.savetxt('framelist.txt',p)
    return p

print datetime.datetime.utcnow()
arrayPath(165, 6, 4)

2017-07-13 09:29:46.438000
Loading Array
Array Loaded
Speed Up Cost Added
Cost Matrix Calculated


C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:58: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


[8.0,
 15.0,
 22.0,
 29.0,
 36.0,
 43.0,
 50.0,
 57.0,
 64.0,
 71.0,
 78.0,
 85.0,
 92.0,
 99.0,
 106.0,
 113.0,
 120.0,
 127.0,
 134.0,
 141.0,
 148.0,
 155.0,
 162]